In [1]:
!pip install hdfs

Processing /home/anj/.cache/pip/wheels/fe/a7/05/23e3699975fc20f8a30e00ac1e515ab8c61168e982abe4ce70/hdfs-2.5.8-cp37-none-any.whl
Processing /home/anj/.cache/pip/wheels/9b/04/dd/7daf4150b6d9b12949298737de9431a324d4b797ffd63f526e/docopt-0.6.2-py2.py3-none-any.whl


In [ ]:
from cdx.cdx_helper import cdx_query
from hdfs import InsecureClient
import zlib

def stream_gzip_decompress(stream):
    dec = zlib.decompressobj(32 + zlib.MAX_WBITS)  # offset 32 to skip the header
    for chunk in stream:
        rv = dec.decompress(chunk)
        if rv:
            yield rv

client = InsecureClient('http://hdfs.api.wa.bl.uk:80', user='access')

for h in cdx_query("http://www.jisc.ac.uk/whatwedo/programmes/programme_preservation/2008sigprops.aspx", limit=100):
    print(h.filename, h.offset, h.length)
    with client.read(h.filename, offset=h.offset, chunk_size=16*1024) as reader:
        for unc in stream_gzip_decompress(reader):
            print(unc.decode('utf-8'))
            break
    


/heritrix/output/frequent-npld/20191203215907/warcs/BL-NPLD-20191207020008229-03586-75~npld-heritrix3-worker-1~8443.warc.gz 634600266 0
